# NAME : KALPESH YASHWANT PATIL

# PRN : 20220802190

# BATCH - DS-1 

# LAB - 03

#TOPIC:- MapReduce Implementation in PySpark

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Amazon_Responded_Oct05.csv to Amazon_Responded_Oct05.csv


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,max #Used in step3(duplicates)

A **Spark session** is the entry point to work with Spark in pyspark. We create it  with the name **"Lab 3 BDA"**, and **getOrCreate()** ensures that we either create a new session or reuse an existing one.

In [ ]:
#Create Spark Session
spark = SparkSession.builder \
        .appName("Lab3 BDA") \
        .getOrCreate()

We read/load the csv file into a Dataframe using **spark.read.csv**

In [ ]:
df = spark.read.csv("Amazon_Responded_Oct05.csv", header=True, inferSchema=True)

In [ ]:
df.printSchema()

root
 |-- id_str: string (nullable = true)
 |-- tweet_created_at: string (nullable = true)
 |-- user_screen_name: string (nullable = true)
 |-- user_id_str: string (nullable = true)
 |-- user_statuses_count: string (nullable = true)
 |-- user_favourites_count: string (nullable = true)
 |-- user_protected: string (nullable = true)
 |-- user_listed_count: string (nullable = true)
 |-- user_following: string (nullable = true)
 |-- user_description: string (nullable = true)
 |-- user_location: string (nullable = true)
 |-- user_verified: string (nullable = true)
 |-- user_followers_count: string (nullable = true)
 |-- user_friends_count: string (nullable = true)
 |-- user_created_at: string (nullable = true)
 |-- tweet_language: string (nullable = true)
 |-- text_: string (nullable = true)
 |-- favorite_count: string (nullable = true)
 |-- favorited: string (nullable = true)
 |-- in_reply_to_screen_name: string (nullable = true)
 |-- in_reply_to_status_id_str: string (nullable = true)
 |--

In [ ]:
df.describe().show()

+-------+--------------------+------------------+--------------------+--------------------+--------------------+---------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------------+-------------------------+-----------------------+--------------------+--------------------+--------------------+
|summary|              id_str|  tweet_created_at|    user_screen_name|         user_id_str| user_statuses_count|user_favourites_count|      user_protected|   user_listed_count|      user_following|    user_description|       user_location|       user_verified|user_followers_count|  user_friends_count|     user_created_at|      tweet_language|               text_|      favorite_count|           favorited|in_reply_to_screen_name|in_reply_to_status_id_st

In [ ]:
df.show(5)

+--------------------+--------------------+----------------+-----------+-------------------+---------------------+--------------+-----------------+--------------+--------------------+-------------+-------------+--------------------+------------------+--------------------+--------------+--------------------+--------------+---------+-----------------------+-------------------------+-----------------------+-------------+---------+--------------------+
|              id_str|    tweet_created_at|user_screen_name|user_id_str|user_statuses_count|user_favourites_count|user_protected|user_listed_count|user_following|    user_description|user_location|user_verified|user_followers_count|user_friends_count|     user_created_at|tweet_language|               text_|favorite_count|favorited|in_reply_to_screen_name|in_reply_to_status_id_str|in_reply_to_user_id_str|retweet_count|retweeted|                text|
+--------------------+--------------------+----------------+-----------+-------------------+--

In [ ]:
df.count() #no.of rows

413247

In [ ]:
len(df.columns)

25

We extract the relavent columns **user_id_str, user_followers_count, text_**

In [ ]:
#Extract the columns
df_selected = df.select("user_id_str", "user_followers_count", "text_")
df_selected.show(5)

+-----------+--------------------+--------------------+
|user_id_str|user_followers_count|               text_|
+-----------+--------------------+--------------------+
|  143515471|                1503|@AmazonHelp Can y...|
|   85741735|              149569|@SeanEPanjab I'm ...|
|  143515471|                1503|@AmazonHelp It wa...|
|  143515471|                1503|@AmazonHelp I am ...|
|   85741735|              149569|@SeanEPanjab Plea...|
+-----------+--------------------+--------------------+
only showing top 5 rows



In [ ]:
#Group by user_id_str and get the max number of followers for each user
df_user_max_followers = df_selected.groupBy("user_id_str").agg(max("user_followers_count").alias("max_followers"))
df_user_max_followers.show(5)

+----------------+--------------------+
|     user_id_str|       max_followers|
+----------------+--------------------+
|            NULL|          sharpewits|
|                |          AmazonHelp|
|      #GalaxyS9 |                   0|
|    #Gaymarriage|@AmazonHelp how d...|
| #NintendoSwitch|                1086|
+----------------+--------------------+
only showing top 5 rows



Since some of the value in the above column "max_followers" are string, we need to preprocess the "user_followers_count" column first


In [ ]:
#first identify and remove invalid values using the filter function
#filer out rows where 'user_followers_count' is not valid number(non-numeric)

df_clean = df_selected.filter(col("user_followers_count").cast("string").rlike("^[0-9]+$"))


In [ ]:
df_clean.show(5)

+-----------+--------------------+--------------------+
|user_id_str|user_followers_count|               text_|
+-----------+--------------------+--------------------+
|  143515471|                1503|@AmazonHelp Can y...|
|   85741735|              149569|@SeanEPanjab I'm ...|
|  143515471|                1503|@AmazonHelp It wa...|
|  143515471|                1503|@AmazonHelp I am ...|
|   85741735|              149569|@SeanEPanjab Plea...|
+-----------+--------------------+--------------------+
only showing top 5 rows



In [ ]:
#Group by user_id_str and get the max number of followers for each user
df_user_max_followers = df_clean.groupBy("user_id_str").agg(max("user_followers_count").alias("max_followers"))
df_user_max_followers.show(5)

+------------------+-------------+
|       user_id_str|max_followers|
+------------------+-------------+
|        #GalaxyS9 |            0|
|   #NintendoSwitch|         1086|
|            #bikes|            0|
|            #manga|            0|
| (distance)ledelse|            0|
+------------------+-------------+
only showing top 5 rows



Here we count number of tweets per user

In [ ]:
#Count the number of tweets per user
df_tweets_per_user = df_selected.groupBy("user_id_str").count()
df_tweets_per_user.show(5)

+-----------+-----+
|user_id_str|count|
+-----------+-----+
| 3196213653|    2|
| 1210875679|    4|
|  163148814|    4|
|  234641258|    2|
|   60760606|    6|
+-----------+-----+
only showing top 5 rows



Now we create a filter to find popular users who have more than 5000 followers

In [ ]:
df_popular_users = df_user_max_followers.filter(col("max_followers") > 5000)
df_popular_users.show(10)

+-----------+-------------+
|user_id_str|max_followers|
+-----------+-------------+
|          4|     20793816|
|          1|     85741735|
|         10|     85741735|
|        100|     85741735|
| 1000295179|        70991|
|       1001|     85741735|
|       1002|     85741735|
|      10021|     85741735|
|  100284780|        37822|
|       1003|     85741735|
+-----------+-------------+
only showing top 10 rows



count words frequency of the tweets

In [ ]:
#Joining the popular users' Dataframe with the original Dataframe
df_popular_tweets = df_selected.join(df_popular_users, "user_id_str")
df_popular_tweets.show(5)

+-----------+--------------------+--------------------+-------------+
|user_id_str|user_followers_count|               text_|max_followers|
+-----------+--------------------+--------------------+-------------+
| 1210875679|               27156|@amazonIN payment...|        27156|
| 1210875679|               27156|@AmazonHelp even ...|        27156|
| 1210875679|               27156|@AmazonHelp super...|        27156|
| 1210875679|               27156|@AmazonHelp How t...|        27156|
|   16279527|               92724|Hi @AmazonHelp ha...|        92724|
+-----------+--------------------+--------------------+-------------+
only showing top 5 rows



In [ ]:
from pyspark.sql.functions import explode, split, lower
#Tokenize the text column into words and explode to get each word on a seperate row
words_df = df_popular_tweets.withColumn("word",explode(split(col("text_"),r"\s+")))

# Remove any non-alphebatic words and lower-case all the words
words_df = words_df.filter(col("word").rlike("^[a-zA-Z]+$"))
words_df = words_df.withColumn("word", lower(col("word")))
words_df.show(5)

+-----------+--------------------+--------------------+-------------+-------+
|user_id_str|user_followers_count|               text_|max_followers|   word|
+-----------+--------------------+--------------------+-------------+-------+
| 1210875679|               27156|@amazonIN payment...|        27156|payment|
| 1210875679|               27156|@amazonIN payment...|        27156|     is|
| 1210875679|               27156|@amazonIN payment...|        27156|    not|
| 1210875679|               27156|@amazonIN payment...|        27156|getting|
| 1210875679|               27156|@amazonIN payment...|        27156|  never|
+-----------+--------------------+--------------------+-------------+-------+
only showing top 5 rows



In [ ]:
# Count the frequency of each word
word_counts = words_df.groupBy("word").count()

# Get the top 10 most popular words by frequency
top_10_words = word_counts.orderBy(col("count").desc()).limit(10)

# Show the result
top_10_words.show()

+------+------+
|  word| count|
+------+------+
|   the|120445|
|    to|120124|
|   you| 88429|
|   for| 56783|
|  your| 52434|
|    us| 49105|
|please| 46465|
|    we| 45592|
| sorry| 44550|
|  this| 38347|
+------+------+

